# Hi :)

----

# Import libraries

In [1]:
import numpy as np
import pandas as pd

# Loading the dataset

In [2]:
# Loading the dataset
SisFall_Dataset= pd.read_csv('Dataset/SisFall_dataset.csv')
# Showing 5 random rows
SisFall_Dataset.sample(5, random_state= 48)

,ADXL345_x,ADXL345_y,ADXL345_z,ITG3200_x,ITG3200_y,ITG3200_z,MMA8451Q_x,MMA8451Q_y,MMA8451Q_z,Situation
636970,3,-237,-149,-120,-122,-96,-78,-881,-437,Fall
734642,-22,59,-283,-43,18,-4,-181,259,-979,Fall
1069635,37,-348,-103,702,-766,437,133,-1329,-305,Not Fall
793919,279,41,-3,-28,46,-10,982,172,153,Fall
541529,7,-429,-144,-295,960,240,-64,-1627,-429,Fall


In [3]:
# Making a copy of original dataset to work with
df= SisFall_Dataset.copy()

In [4]:
# Checking the number of rows and columns
df.shape

(2097150, 10)

In [5]:
df.shape[0] / 150

13981.0

In [6]:
df.columns

Index(['ADXL345_x', 'ADXL345_y', 'ADXL345_z', 'ITG3200_x', 'ITG3200_y',
       'ITG3200_z', 'MMA8451Q_x', 'MMA8451Q_y', 'MMA8451Q_z', 'Situation'],
      dtype='object')

# Kalman Filter

Kalman Filter implementation on one axis

In [ ]:
# def run_kalman_filter(measurements, R_value, A, H, Q, x0, P0):
#     x= x0.copy()
#     P= P0.copy()
#     estimates= []

#     for z in measurements:
#         # Predict
#         x= A @ x
#         P= A @ P @ A.T + Q

#         # Update
#         y= np.array(z).reshape(-1, 1) - H @ x
#         S= H @ P @ H.T + R_value
#         K= P @ H.T @ np.linalg.inv(S)
#         x= x + K @ y
#         P= (np.eye(P.shape[0]) - K @ H) @ P

#         estimates.append(x[0, 0])  # Store estimated state

#     return estimates

In [ ]:
# A= np.array([[1.0]])           # Identity if you're not modeling dynamics
# H= np.array([[1.0], [1.0], [1.0]])  # Each sensor observes the same state
# Q= np.array([[0.001]])         # Process noise
# R= np.diag([0.05, 0.05, 0.1])  # Sensor noise: gyro might be less reliable
# x0= np.array([[0.0]])          # Initial state
# P0= np.array([[1.0]])          # Initial covariance

In [ ]:
# measurements= df[['ADXL345_x', 'ITG3200_x', 'MMA8451Q_x']].values

In [ ]:
# estimates= run_kalman_filter(measurements, R, A, H, Q, x0, P0)

In [ ]:
# len(estimates)

2097150

Kalman Filter Implementation on three axis

In [12]:
def run_kalman_filter_vector(measurements, R_value, A, H, Q, x0, P0):
    x= x0.copy()
    P= P0.copy()
    estimates= []

    for z in measurements:
        # Predict
        x= A @ x
        P= A @ P @ A.T + Q

        # Update
        y= np.array(z).reshape(-1, 1) - H @ x
        S= H @ P @ H.T + R_value
        K= P @ H.T @ np.linalg.inv(S)
        x= x + K @ y
        P= (np.eye(P.shape[0]) - K @ H) @ P

        estimates.append(x.flatten())  # Store estimated state

    return estimates

In [13]:
A= np.eye(3)   # No dynamics for now
Q= np.eye(3) * 0.001
x0= np.zeros((3, 1))
P0= np.eye(3)

# Same H matrix as above
H= np.array([
    [1, 0, 0], [0, 1, 0], [0, 0, 1],  # accel1
    [1, 0, 0], [0, 1, 0], [0, 0, 1],  # accel2
    [1, 0, 0], [0, 1, 0], [0, 0, 1],  # gyro
])

R= np.diag([0.05]*6 + [0.1]*3)  # More trust in accelerometers

In [14]:
measurements= df[['ADXL345_x', 'ADXL345_y', 'ADXL345_z', 'MMA8451Q_x', 'MMA8451Q_y', 'MMA8451Q_z', 'ITG3200_x', 'ITG3200_y', 'ITG3200_z']].values

In [15]:
estimates= run_kalman_filter_vector(measurements, R, A, H, Q, x0, P0)

In [16]:
df[['x_kalman', 'y_kalman', 'z_kalman']]=np.array(estimates).round(2)
df= df[[col for col in df.columns if col != 'Situation'] + ['Situation']]

In [17]:
df.head()

,ADXL345_x,ADXL345_y,ADXL345_z,ITG3200_x,ITG3200_y,ITG3200_z,MMA8451Q_x,MMA8451Q_y,MMA8451Q_z,x_kalman,y_kalman,z_kalman,Situation
0,5,-234,-82,37,4,-7,9,-959,-319,12.75,-467.07,-158.63,Fall
1,2,-234,-87,35,4,-7,8,-964,-319,11.86,-472.82,-161.25,Fall
2,6,-234,-84,35,3,-7,11,-962,-323,12.55,-474.60,-162.31,Fall
3,5,-234,-85,34,4,-8,7,-961,-323,12.28,-475.35,-163.03,Fall
4,7,-237,-83,34,4,-7,9,-959,-323,12.51,-475.92,-163.23,Fall
